In [1]:
# Run this notebook with a virtual environment that has NNUnet V1 installed


import sys
import os
import shutil
sys.path.append('/home/edwardsb/repositories/be-SATGOpenFL/openfl/federated/task')
# older script
from fedsim_setup import main as setup_fedsim
from fl_setup import main as setup_fl
from nnunet_v1_with_old_plans_name_hardcoded import train_nnunet
from nnunet.paths import default_plans_identifier

network = '3d_fullres'
network_trainer = 'nnUNetTrainerV2'
fold = '0'

cuda_device='5'

os.environ['CUDA_VISIBLE_DEVICES']=cuda_device

def train_on_task(task, continue_training=True, current_epoch=0, without_data_unpacking=False, use_compressed_data=False):
    print(f"###########\nStarting training for task: {task}\n")
    train_nnunet(epochs=1, 
                 current_epoch = current_epoch, 
                 network = network,
                 task=task, 
                 network_trainer = network_trainer, 
                 fold=fold, 
                 continue_training=continue_training, 
                 use_compressed_data=use_compressed_data)
    
def rm_tree(path):
    if os.path.exists(path):
        shutil.rmtree(path)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [2]:
# NOTE: This cell takes about 8 minutes to run on spr01 (Titan XP)

# First do what we did before

# Some paths below will be under my folders, but someone else might want to replace this part
test_pardir = '/raid/edwardsb/projects/RANO'


datadir_568 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_preprocessed', 'Task568_MultPathTest')
# next two for deleting only
alt_1_datadir_568 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_raw_data', 'Task568_MultPathTest')
alt_2_datadir_568 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_cropped_data', 'Task568_MultPathTest')

model_pre_dir_568 = os.path.join(test_pardir, 'NNUnetModels', 'nnUNet', '3d_fullres', 'Task568_MultPathTest')
modeldir_568 = os.path.join(model_pre_dir_568, 'nnUNetTrainerV2__nnUNetPlansv2.1', 'fold_0')

datadir_569 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_preprocessed', 'Task569_MultPathTest')
# next two for deleting only
alt_1_datadir_569 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_raw_data', 'Task569_MultPathTest')
alt_2_datadir_569 = os.path.join(test_pardir, 'BraTS22_pretending_tobe_postopp', 'nnUNet_raw_data_base', 'nnUNet_cropped_data', 'Task569_MultPathTest')

model_pre_dir_569 = os.path.join(test_pardir, 'NNUnetModels', 'nnUNet', '3d_fullres', 'Task569_MultPathTest')
model_dir_569 = os.path.join(model_pre_dir_569, 'nnUNetTrainerV2__nnUNetPlansv2.1', 'fold_0')

# At this point deleted some folders via linux command line

rm_tree(datadir_568)
rm_tree(alt_1_datadir_568)
rm_tree(alt_2_datadir_568)
rm_tree(model_pre_dir_568)


rm_tree(datadir_569)
rm_tree(alt_1_datadir_569)
rm_tree(alt_2_datadir_569)
rm_tree(model_pre_dir_569)


setup_fedsim(postopp_pardirs=['/raid/edwardsb/projects/RANO/test_data_links_random_times_0'], 
         first_three_digit_task_num=568,
         init_model_path=None,
         init_model_info_path= None, 
         task_name='MultPathTest', 
         percent_train=0.8, 
         split_logic='by_subject_time_pair', 
         network='3d_fullres', 
         network_trainer='nnUNetTrainerV2', 
         fold=0,  
         timestamp_selection='all', 
         num_institutions=1, 
         cuda_device='5', 
         verbose=False)

# The source for the init model paths are located in the 568 results folder
init_model_path_568 = os.path.join(modeldir_568, 'model_initial_checkpoint.model') 
init_model_info_path_568 =  os.path.join(modeldir_568, 'model_initial_checkpoint.model.pkl') 

# The plan paths are under the data folders
plan_path_568 = os.path.join(datadir_568, 'nnUNetPlansv2.1_plans_3D.pkl')
plan_path_569 = os.path.join(datadir_569, 'nnUNetPlansv2.1_plans_3D.pkl')

# Now set up 569 as we did in the first fed test, with the old script that passes only initial model and initial model info bu only shares plan across the first collaborator
# in the call with the others (though there are no others here)

setup_fedsim(postopp_pardirs=['/raid/edwardsb/projects/RANO/test_data_links_random_times_1'], 
         first_three_digit_task_num=569,
         init_model_path=init_model_path_568,
         init_model_info_path= init_model_info_path_568,
         task_name='MultPathTest', 
         percent_train=0.8, 
         split_logic='by_subject_time_pair', 
         network='3d_fullres', 
         network_trainer='nnUNetTrainerV2', 
         fold=0,  
         timestamp_selection='all', 
         num_institutions=1, 
         cuda_device='5', 
         verbose=False)

# Now try to train and see we get a shape mismatch (note I need to replace the new default value for p below as before we used the default plans identifier but now the code useds a POSTOPP specific string)
train_on_task(task='Task569_MultPathTest', current_epoch=1)


######### CREATING SYMLINKS TO POSTOPP DATA FOR COLLABORATOR 0 #########


######### GENERATING DATA JSON FILE FOR COLLABORATOR 0 #########


######### OS CALL TO PREPROCESS DATA FOR COLLABORATOR 0 #########



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Verifying training set
checking case BraTS2021_00468_2008.03.26
checking case BraTS2021_00512_2008.03.26
checking case BraTS2021_00512_2008.12.11
checking case BraTS2021_00565_2008.03.26
checking case BraTS2021_01277_2008.03.26
Verifying label values
Expected label values are [0, 1, 2, 3, 4]
Labels OK
Dataset OK
BraTS2021_00468_2008.03.26
BraTS2021_00512_2008.03.26
BraTS2021_00512_2008.12.11
BraTS2021_00565_2008.03.26
BraTS2

RuntimeError: Error(s) in loading state_dict for Generic_UNet:
	size mismatch for tu.0.weight: copying a param with shape torch.Size([320, 320, 2, 2, 1]) from checkpoint, the shape in current model is torch.Size([320, 320, 2, 2, 2]).

In [5]:
# Now drop in the plans file that should have been shared and see that training works now

shutil.copyfile(src=plan_path_568,dst=plan_path_569)

# Now try to train again and this time see it works
train_on_task(task='Task569_MultPathTest', current_epoch=1)

###########
Starting training for task: Task569_MultPathTest

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  4
modalities:  {0: '_0000', 1: '_0001', 2: '_0002', 3: '_0003'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT'), (1, 'nonCT'), (2, 'nonCT'), (3, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [5, 5, 4], 'patch_size': array([128, 160, 112]), 'median_patient_size_in_voxels': array([137, 171, 130]), 'current_spacing': array([1., 1., 1.]), 'original_spacing': array([1., 1., 1.]), 'do_dummy_2D_data_aug': False, 'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 1]], 'co

done
2024-06-03 12:03:43.852579: loading checkpoint /raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task569_MultPathTest/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_final_checkpoint.model train= True
2024-06-03 12:03:44.302680: lr: 0.009991
using pin_memory on device 0
using pin_memory on device 0
2024-06-03 12:03:56.958957: Unable to plot network architecture:
2024-06-03 12:03:56.968052: No module named 'hiddenlayer'
2024-06-03 12:03:56.968819: 
printing the network instead:

2024-06-03 12:03:56.969363: Generic_UNet(
  (conv_blocks_localization): ModuleList(
    (0): Sequential(
      (0): StackedConvLayers(
        (blocks): Sequential(
          (0): ConvDropoutNormNonlin(
            (conv): Conv3d(640, 320, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (instnorm): InstanceNorm3d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
            (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
          )
        )
      )

/home/edwardsb/repositories/nnUNet/nnunet/training/network_training/nnUNetTrainer.py:719: RuntimeWarning: invalid value encountered in scalar divide
  global_dc_per_class = [i for i in [2 * i / (2 * i + j + k) for i, j, k in


2024-06-03 12:09:41.556391: lr: 0.009982
2024-06-03 12:09:41.557259: This epoch took 344.577959 s

2024-06-03 12:09:41.641403: saving checkpoint to /raid/edwardsb/projects/RANO/NNUnetModels/nnUNet/3d_fullres/Task569_MultPathTest/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_final_checkpoint.model...
2024-06-03 12:09:42.202568: done, saving took 0.64 seconds


In [ ]:
# To see sucessful data processing and model definitions with the new scripts, run the sript: test_fl_setup.py in the same folder holding this notebook.